## Key pair generation

## Sign

- M message
- A, the signer, pick a random $k, 1 < k < n$ and calculate $R = kG$
- A calculate $e = h(R || M)$
- A calculate $s = (k + de) mod n$
- The signature is the pair $(R, s)$ 

## Verify

- B, the receiver, calculates $R_{v} = sG$
- B calculats $e = h(R || M)$
- if $R_{v} - R = eP$ the signature is valid